In [2]:
import os
import tensorflow as tf
import numpy as np
print(tf.__version__)
print(np.__version__)

1.12.0
1.15.4


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True

In [4]:
# without batch_size

feed_a = np.array([1,2,3,6,4])
feed_b = np.array([1,0,0,5,2])
input_a = tf.constant(feed_a, dtype=tf.float32)
input_b = tf.constant(feed_b, dtype=tf.int32)
length = tf.shape(input_a)[0]
array_size = tf.reduce_sum(input_b)
output_c = tf.TensorArray(dtype=np.float32, size=array_size, clear_after_read=False)

i = 0
def cond_outter(input_a, input_b, output_c, i):
    return tf.less(i, length)

def loop_outter(input_a, input_b, output_c, i):
    j = 0
    def cond_inner(input_a, input_b, output_c, i, j):
        return tf.less(j, input_b[i])

    def loop_inner(input_a, input_b, output_c, i, j):
        output_c = output_c.write(tf.reduce_sum(input_b[:i]) + j, input_a[i])
        return input_a, input_b, output_c, i, j+1


    
    _, _, output_c, _, _ = tf.while_loop(cond=cond_inner,
                                         body=loop_inner,
                                         loop_vars=[input_a, input_b, output_c, i, j])
    return input_a, input_b, output_c, i+1

# result
_, _, output_c, _ = tf.while_loop(cond=cond_outter, 
                                     body=loop_outter, 
                                     loop_vars=[input_a, input_b, output_c, i])

c=output_c.stack()

with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(c)
    print(c.eval())

[1. 6. 6. 6. 6. 6. 4. 4.]


In [4]:
# with batch_size

feed_a = np.array([[[1,1],[2,2],[3,3],[4,4]],[[2,2],[3,3],[4,4],[5,5]]])
feed_b = np.array([[1,0,0,5],[2,0,4,3]])
input_a = tf.constant(feed_a, dtype=tf.float32)
input_b = tf.constant(feed_b, dtype=tf.int32)

batch_len = tf.shape(input_a)[0]
batch_len = tf.cast(batch_len, tf.int32)
max_len = tf.reduce_max(tf.reduce_sum(input_b, axis=-1))
max_len = tf.cast(max_len, tf.int32)
array_size = tf.cast(batch_len*max_len, tf.int32)

output_c = tf.TensorArray(dtype=np.float32, size=array_size, clear_after_read=False)

batch_index = 0
def cond_outter(input_a, input_b, output_c, batch_index, max_len):
    return tf.less(batch_index, batch_len)

def loop_outter(input_a, input_b, output_c, batch_index, max_len):
    frame = input_a[batch_index]
    frame_repeat = input_b[batch_index]
    
    frame_index = 0
    frame_len = tf.shape(frame)[0]
    frame_len = tf.cast(frame_len, tf.int32)
    def cond_inner(input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len):
        return tf.less(frame_index, frame_len)

    def loop_inner(input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len):
        content = frame[frame_index]
        time = 0
        repeats = frame_repeat[frame_index]
        repeats = tf.cast(repeats, tf.int32)
        def cond(input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len, time, repeats):
            return tf.less(time, repeats)
        def loop(input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len, time, repeats):
            location_batch = batch_index * max_len
            location_frame = tf.cast(tf.reduce_sum(frame_repeat[:frame_index]), tf.int32)
            location = location_batch + location_frame + time
            output_c = output_c.write(location, content)
            return input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len, time+1, repeats
    
        _, _, output_c, _, _, _, _, _, _, _, _ = tf.while_loop(cond,loop,
          [input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len, time, repeats])
        return input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index+1, frame_len

    _, _, output_c, _, _, _, _, _, _ = tf.while_loop(cond_inner,loop_inner, 
      [input_a, input_b, output_c, batch_index, max_len, frame, frame_repeat, frame_index, frame_len])
    return input_a, input_b, output_c, batch_index+1, max_len

_, _, output_c, _, _= tf.while_loop(cond_outter,loop_outter, 
  [input_a, input_b, output_c, batch_index, max_len])

c=output_c.stack()
c=tf.reshape(c, [batch_len, -1, 2])

with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(c)
    print(c.eval())

[[[1. 1.]
  [4. 4.]
  [4. 4.]
  [4. 4.]
  [4. 4.]
  [4. 4.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[2. 2.]
  [2. 2.]
  [4. 4.]
  [4. 4.]
  [4. 4.]
  [4. 4.]
  [5. 5.]
  [5. 5.]
  [5. 5.]]]
